In [2]:
import re
import pandas as pd
import numpy as np
import nltk

### Load in all transcripts with meta data

In [3]:
# load meta data of talks
df_meta = pd.read_csv('./data/ted_main.csv')
df_meta.head(1)

,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,ratings,related_talks,speaker_occupation,tags,title,url,views
0,4553,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,1140825600,60,Ken Robinson,Ken Robinson: Do schools kill creativity?,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...","[{'id': 865, 'hero': 'https://pe.tedcdn.com/im...",Author/educator,"['children', 'creativity', 'culture', 'dance',...",Do schools kill creativity?,https://www.ted.com/talks/ken_robinson_says_sc...,47227110


In [4]:
# load full transcripts of talks
df_scripts = pd.read_csv('./data/transcripts.csv')
df_scripts.head(1)

,transcript,url
0,Good morning. How are you?(Laughter)It's been ...,https://www.ted.com/talks/ken_robinson_says_sc...


In [25]:
df_meta[df_meta.title=='The boost students need to overcome obstacles']

,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,ratings,related_talks,speaker_occupation,tags,title,url,views


In [20]:
# join meta data with transcripts for all talks
df_all = df_meta.merge(df_scripts, on='url')
print(len(df_all))
df_all.head(1)

2467


,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,ratings,related_talks,speaker_occupation,tags,title,url,views,transcript
0,4553,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,1140825600,60,Ken Robinson,Ken Robinson: Do schools kill creativity?,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...","[{'id': 865, 'hero': 'https://pe.tedcdn.com/im...",Author/educator,"['children', 'creativity', 'culture', 'dance',...",Do schools kill creativity?,https://www.ted.com/talks/ken_robinson_says_sc...,47227110,Good morning. How are you?(Laughter)It's been ...


In [23]:
N = 146
print(df_all.name[N])
print(df_scripts.transcript[N])

Kenichi Ebina: My magic moves
(Applause)(Music)(Applause)


In [66]:
# select talks tagged with "education"
df_edu = df_all[df_all['tags'].apply(lambda x: 'education' in x)]
print(len(df_edu))
df_edu.head(1)

147


,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,ratings,related_talks,speaker_occupation,tags,title,url,views,transcript
0,4553,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,1140825600,60,Ken Robinson,Ken Robinson: Do schools kill creativity?,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...","[{'id': 865, 'hero': 'https://pe.tedcdn.com/im...",Author/educator,"['children', 'creativity', 'culture', 'dance',...",Do schools kill creativity?,https://www.ted.com/talks/ken_robinson_says_sc...,47227110,Good morning. How are you?(Laughter)It's been ...


### Load scraped data

In [26]:
df_scrape = pd.read_pickle('./data/TED_scraped.pkl')
df_scrape.head()

,_id,link,speaker,title,transcript
0,5aa5633ed659da7c9b7748ab,/talks/petter_johansson_do_you_really_know_why...,Petter Johansson,Do you really know why you do what you do?,So why do you think the rich should pay more i...
1,5aa56340d659da7c9b7748ac,/talks/sophie_andrews_the_best_way_to_help_is_...,Sophie Andrews,The best way to help is often just to listen,"After cutting her arm with a broken glass, she..."
2,5aa56342d659da7c9b7748ad,/talks/musimbi_kanyoro_to_solve_the_world_s_bi...,Musimbi Kanyoro,"To solve the world's biggest problems, invest ...",My mother was a philanthropist. And now I know...
3,5aa56343d659da7c9b7748ae,/talks/simone_bianco_and_tom_zimmerman_the_won...,Simone Bianco and Tom Zimmerman,The wonderful world of life in a drop of water,Tom Zimmerman: We'd like to take you on a fant...
4,5aa56345d659da7c9b7748af,/talks/naomi_klein_how_shocking_events_can_spa...,Naomi Klein,How shocking events can spark positive change,


In [30]:
df_scrape = df_scrape.rename(columns={'speaker':'main_speaker'})

In [33]:
df_scrape = df_scrape.drop(['_id'],axis=1)
df_scrape.head()

,link,main_speaker,title,transcript
0,/talks/petter_johansson_do_you_really_know_why...,Petter Johansson,Do you really know why you do what you do?,So why do you think the rich should pay more i...
1,/talks/sophie_andrews_the_best_way_to_help_is_...,Sophie Andrews,The best way to help is often just to listen,"After cutting her arm with a broken glass, she..."
2,/talks/musimbi_kanyoro_to_solve_the_world_s_bi...,Musimbi Kanyoro,"To solve the world's biggest problems, invest ...",My mother was a philanthropist. And now I know...
3,/talks/simone_bianco_and_tom_zimmerman_the_won...,Simone Bianco and Tom Zimmerman,The wonderful world of life in a drop of water,Tom Zimmerman: We'd like to take you on a fant...
4,/talks/naomi_klein_how_shocking_events_can_spa...,Naomi Klein,How shocking events can spark positive change,


In [39]:
inds = []
for i in range(len(df_scrape)):
    if len(df_scrape.iloc[i].transcript) > 0:
        inds.append(i)
len(inds)

57

In [40]:
df_scrape = df_scrape.iloc[inds]

In [43]:
df_all = pd.concat([df_all,df_scrape])

In [44]:
len(df_all)

2524

In [45]:
df_all.head()

,comments,description,duration,event,film_date,languages,link,main_speaker,name,num_speaker,published_date,ratings,related_talks,speaker_occupation,tags,title,transcript,url,views
0,4553.0,Sir Ken Robinson makes an entertaining and pro...,1164.0,TED2006,1.140826e+09,60.0,NaN,Ken Robinson,Ken Robinson: Do schools kill creativity?,1.0,1.151367e+09,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...","[{'id': 865, 'hero': 'https://pe.tedcdn.com/im...",Author/educator,"['children', 'creativity', 'culture', 'dance',...",Do schools kill creativity?,Good morning. How are you?(Laughter)It's been ...,https://www.ted.com/talks/ken_robinson_says_sc...,47227110.0
1,265.0,With the same humor and humanity he exuded in ...,977.0,TED2006,1.140826e+09,43.0,NaN,Al Gore,Al Gore: Averting the climate crisis,1.0,1.151367e+09,"[{'id': 7, 'name': 'Funny', 'count': 544}, {'i...","[{'id': 243, 'hero': 'https://pe.tedcdn.com/im...",Climate advocate,"['alternative energy', 'cars', 'climate change...",Averting the climate crisis,"Thank you so much, Chris. And it's truly a gre...",https://www.ted.com/talks/al_gore_on_averting_...,3200520.0
2,124.0,New York Times columnist David Pogue takes aim...,1286.0,TED2006,1.140739e+09,26.0,NaN,David Pogue,David Pogue: Simplicity sells,1.0,1.151367e+09,"[{'id': 7, 'name': 'Funny', 'count': 964}, {'i...","[{'id': 1725, 'hero': 'https://pe.tedcdn.com/i...",Technology columnist,"['computers', 'entertainment', 'interface desi...",Simplicity sells,"(Music: ""The Sound of Silence,"" Simon & Garfun...",https://www.ted.com/talks/david_pogue_says_sim...,1636292.0
3,200.0,"In an emotionally charged talk, MacArthur-winn...",1116.0,TED2006,1.140912e+09,35.0,NaN,Majora Carter,Majora Carter: Greening the ghetto,1.0,1.151367e+09,"[{'id': 3, 'name': 'Courageous', 'count': 760}...","[{'id': 1041, 'hero': 'https://pe.tedcdn.com/i...",Activist for environmental justice,"['MacArthur grant', 'activism', 'business', 'c...",Greening the ghetto,If you're here today — and I'm very happy that...,https://www.ted.com/talks/majora_carter_s_tale...,1697550.0
4,593.0,You've never seen data presented like this. Wi...,1190.0,TED2006,1.140566e+09,48.0,NaN,Hans Rosling,Hans Rosling: The best stats you've ever seen,1.0,1.151441e+09,"[{'id': 9, 'name': 'Ingenious', 'count': 3202}...","[{'id': 2056, 'hero': 'https://pe.tedcdn.com/i...",Global health expert; data visionary,"['Africa', 'Asia', 'Google', 'demo', 'economic...",The best stats you've ever seen,"About 10 years ago, I took on the task to teac...",https://www.ted.com/talks/hans_rosling_shows_t...,12005869.0


### Basic processing on the transcripts

In [46]:
docs_all_raw = list(df_all['transcript'])
# docs_edu_raw = list(df_edu['transcript'])

In [47]:
len(docs_all_raw[0])

17409

In [63]:
raw_text = docs_all_raw[1000]
print(raw_text)

So today, I would like to talk with you about bionics, which is the popular term for the science of replacing part of a living organism with a mechatronic device, or a robot. It is essentially the stuff of life meets machine. And specifically, I'd like to talk with you about how bionics is evolving for people with arm amputations.This is our motivation. Arm amputation causes a huge disability. I mean, the functional impairment is clear. Our hands are amazing instruments. And when you lose one, far less both, it's a lot harder to do the things we physically need to do. There's also a huge emotional impact. And actually, I spend as much of my time in clinic dealing with the emotional adjustment of patients as with the physical disability. And finally, there's a profound social impact. We talk with our hands. We greet with our hands. And we interact with the physical world with our hands. And when they're missing, it's a barrier. Arm amputation is usually caused by trauma, with things lik

In [58]:
new_text = cleanup(raw_text, 'lemma')
print(new_text)

it been great hasnt it. ive been blown away by the whole thing. in fact im leaving. there have been three theme running through the conference which are relevant to what i want to talk about. one is the extraordinary evidence of human creativity in all of the presentation that weve had and in all of the people here. just the variety of it and the range of it. the second is that it put u in a place where we have no idea whats going to happen in term of the future. no idea how this may play out. i have an interest in education. actually what i find is everybody ha an interest in education. dont you. i find this very interesting. if youre at a dinner party and you say you work in education actually youre not often at dinner party frankly. if you work in education youre not asked. and youre never asked back curiously. thats strange to me. but if you are and you say to somebody you know they say what do you do. and you say you work in education you can see the blood run from their face. the

* Remove non-talk words, such as descriptions of audience interaction "(Applause)" "(Laughter)"
* Condense all whitespaces
* Remove non-alpha chars
* Use lemmatizer/stemmer

In [48]:
from nltk.tokenize import sent_tokenize

def cleanup(text, stem_method):
    
    # remove things like '(Applause)', '(Laughter)', etc.
    new_text = re.sub(r'\([\w ]+\)', r'', text)
    
    # remove first two and last one sentence
    new_text = new_text.replace('.','. ')
    new_text = new_text.replace('?','? ')
    new_text = new_text.replace('!','! ')
    new_text = new_text.replace('"','" ')
    sent_list = sent_tokenize(new_text)[2:-1]
    
    # for each sentence, condense all whitespace and remove non-alpha chars
    new_sent_list = []
    for sent in sent_list:
        content = re.sub('\s+', ' ', sent.lower())  # condense all whitespace
        content = re.sub('[^A-Za-z ]+', '', content)  # remove non-alpha chars
    
        # lemmatize words
        words = content.split()
        if stem_method == 'lemma':
            stemmer = nltk.stem.WordNetLemmatizer()
            words = [stemmer.lemmatize(word) for word in words]
        elif stem_method == 'porter':
            stemmer = nltk.stem.PorterStemmer()
            words = [stemmer.stem(word) for word in words]
        new_sent = ' '.join(words)
        new_sent_list.append(new_sent)
    
    return '. '.join(new_sent_list)

In [49]:
# docs_edu_porter = [cleanup(doc,'porter') for doc in docs_edu_raw]
docs_all_porter = [cleanup(doc,'porter') for doc in docs_all_raw]
# docs_edu_lemma = [cleanup(doc,'lemma') for doc in docs_edu_raw]
docs_all_lemma = [cleanup(doc,'lemma') for doc in docs_all_raw]

In [51]:
# docs_all_lemma[1]

In [53]:
# df = df_all.iloc[:,:]
# df['transcript'] = docs_all_lemma
# df.head()

In [54]:
# df.to_pickle('./data/df_all_lemma.pkl')

### EDA: LDA with TF terms, NMF with TF-IDF terms

In [46]:
docs = docs_edu_lemma

In [47]:
# Author: Olivier Grisel <olivier.grisel@ensta.org>
#         Lars Buitinck
#         Chyi-Kwei Yau <chyikwei.yau@gmail.com>
# License: BSD 3 clause

from time import time

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

n_samples = 2000
n_features = 1000
n_components = 15
n_top_words = 10
n_gram = 1
alpha = 0.1
stop_choice= 'english'#stop_list[:100]

max_df = 0.3
min_df = 3
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()


# Use tf-idf features for NMF.
print("Extracting tf-idf features for NMF...")
tfidf_vectorizer = TfidfVectorizer(max_df=max_df, min_df=min_df,
                                   ngram_range=(n_gram,n_gram),
                                   max_features=n_features,
                                   stop_words=stop_choice)
t0 = time()
tfidf = tfidf_vectorizer.fit_transform(docs)
print("done in %0.3fs." % (time() - t0))

# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=max_df, min_df=min_df,
                                ngram_range=(n_gram,n_gram),
                                max_features=n_features,
                                stop_words=stop_choice)
t0 = time()
tf = tf_vectorizer.fit_transform(docs)
print("done in %0.3fs." % (time() - t0))
print()

# Fit the NMF model
print("Fitting the NMF model (Frobenius norm) with tf-idf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_components, random_state=1,
          alpha=alpha, l1_ratio=.5).fit(tfidf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in NMF model (Frobenius norm):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

# Fit the NMF model
print("Fitting the NMF model (generalized Kullback-Leibler divergence) with "
      "tf-idf features, n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_components, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=alpha,
          l1_ratio=.5).fit(tfidf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in NMF model (generalized Kullback-Leibler divergence):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

print("Fitting LDA models with tf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
lda = LatentDirichletAllocation(n_components=n_components, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
t0 = time()
lda.fit(tf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Extracting tf-idf features for NMF...
done in 0.191s.
Extracting tf features for LDA...
done in 0.212s.

Fitting the NMF model (Frobenius norm) with tf-idf features, n_samples=2000 and n_features=1000...
done in 0.091s.

Topics in NMF model (Frobenius norm):
Topic #0: art grade math stuff public information america adult challenge business
Topic #1: video online interactive feedback lecture interaction camera peer youtube discussion
Topic #2: plastic bag water ocean local plant nuclear collect significant map
Topic #3: english india slum delhi language village experiment grandmother indian cloud
Topic #4: food lunch cook meal plant eat sugar dollar farmer feeding
Topic #5: game player team pie reward video brain playing task win
Topic #6: baby ball pregnant evidence hypothesis toy broccoli brain mother blue
Topic #7: project opensource code robot scratch bird engineering block game tool
Topic #8: daughter father afghanistan men village mother sister refugee brother training
Topic #9: m

### Methodology conclusion
* Use TF-IDF with NMF (Frobenius norm) gives the best coherent topic (in both cases: all talks and edu talks)
* Lemmatizer works better than stemmer
* However, lemmatizer does not distinguish verbs in different tenses
* Consider Word2Vec to combine terms into groups, eg. {africa, african}
* Need to refine stop-word list

In [65]:
from functools import reduce
list1 = list(range(2495,2501))
prod1 = reduce(lambda x,y:x*y,list1)
list2 = list(range(2474,2480))
prod2 = reduce(lambda x,y:x*y,list2)

1 - prod2/prod1

0.04940176816749564